## Agent with web-search tool

In [1]:
from __future__ import annotations

import os
import time
import hashlib
from dataclasses import dataclass
from typing import Any, Optional, List, Dict

import requests
from pydantic import BaseModel, HttpUrl, Field
from pydantic_ai import Agent, RunContext
from openai import OpenAI #AsyncOpenAI
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider

In [2]:
# Load necessary API keys
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# -----------------------------
# Output models (tool returns this)
# -----------------------------

class BraveResult(BaseModel):
    title: str
    url: HttpUrl
    description: Optional[str] = None


class WebDocument(BaseModel):
    url: HttpUrl
    title: str = ""
    description: Optional[str] = None
    text: str
    fetched_at_utc: str
    sha256: str
    meta: Dict[str, Any] = Field(default_factory=dict)


class WebSearchAndReadOutput(BaseModel):
    query: str
    search_results: List[BraveResult]
    documents: List[WebDocument]
    errors: List[Dict[str, str]]

In [4]:
# -----------------------------
# Small utilities
# -----------------------------

def utc_now_iso() -> str:
    return time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())

def sha256_text(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8", errors="ignore")).hexdigest()

def _truncate(s: str, max_chars: int) -> str:
    if max_chars <= 0:
        return s
    return s[:max_chars]

In [5]:
# -----------------------------
# Brave Web Search client
# -----------------------------

class BraveSearchClient:
    def __init__(self, api_key: str, session: Optional[requests.Session] = None, timeout_s: int = 30):
        # Brave Search API subscription token (sent in X-Subscription-Token header).
        self.api_key = api_key

        # Reuse a requests.Session if provided (connection pooling = faster + fewer TCP handshakes).
        # If none provided, create one.
        self.session = session or requests.Session()

        # Timeout (seconds) for the HTTP request to Brave.
        self.timeout_s = timeout_s

        # Brave Web Search endpoint.
        self.endpoint = "https://api.search.brave.com/res/v1/web/search"

    def search(self, query: str, count: int = 10) -> List[Dict[str, str]]:
        # Headers required by Brave:
        # - X-Subscription-Token authenticates the request
        # - Accept says we want JSON back
        # - Accept-Encoding gzip allows compressed responses (smaller / faster)
        headers = {
            "X-Subscription-Token": self.api_key,
            "Accept": "application/json",
            "Accept-Encoding": "gzip",
        }

        # Query parameters:
        # - q: the user query string
        # - count: number of results requested (Brave may return fewer)
        params = {"q": query, "count": count}

        # Send GET request to Brave.
        r = self.session.get(
            self.endpoint,
            params=params,
            headers=headers,
            timeout=self.timeout_s,
        )

        # Raise an exception for HTTP errors 
        r.raise_for_status()

        # Parse JSON response body into a Python dict
        data = r.json()

        # "Defensive parsing":
        # Instead of assuming data["web"]["results"] always exists (which could KeyError),
        # we safely navigate the dict:
        # - data.get("web") returns None if missing, so we replace with {}
        # - then .get("results") returns None if missing, so we replace with []
        results = (data.get("web") or {}).get("results") or []

        # Convert Brave's richer result objects into a small, stable format
        # your agent/tool can rely on.
        out: List[Dict[str, str]] = []
        for item in results:
            url = item.get("url")
            title = item.get("title") or ""
            desc = item.get("description") or ""

            # Only keep results that have a URL
            if url:
                out.append({"url": url, "title": title, "description": desc})

        # Return a list of dicts like:
        # [{"url": "...", "title": "...", "description": "..."}, ...]
        return out

In [6]:
# -----------------------------
# Jina Webpage Reader client
# -----------------------------

class JinaReaderClient:
    def __init__(self, api_key: str, timeout_s: int = 60):
        # Choose the Jina Reader base URL.
        self.base = "https://r.jina.ai"
        #self.base = "https://eu.r.jina.ai" if eu else "https://r.jina.ai"

        # API key used for Authorization: Bearer <key>
        # (lets you access authenticated features / higher limits depending on Jina plan)
        self.api_key = api_key

        # How long we wait (in seconds) before giving up on the HTTP request.
        self.timeout_s = timeout_s

    def read_url(self, url: str, use_readerlm_v2: bool = False) -> dict:
        # HTTP headers for the Jina Reader POST request:
        # - Request JSON response
        # - Send a JSON body
        # - Authenticate with Bearer token
        headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}",
        }

        # Optional: ask Jina to use ReaderLM-v2 for higher-quality extraction.
        # This can improve results on complex pages, but may be slower / costlier.
        if use_readerlm_v2:
            headers["X-Respond-With"] = "readerlm-v2"

        # Make the request:
        # POST <base>/ with JSON {"url": "..."}
        r = requests.post(
            f"{self.base}/",
            headers=headers,
            json={"url": url},
            timeout=self.timeout_s,
        )

        # If status code is 4xx/5xx, raise an exception (fail fast, easier debugging).
        r.raise_for_status()

        # Parse and return the JSON response as a Python dict.
        # Typical shape is something like {"data": {...}, "status": ..., "code": ...}
        return r.json()

In [7]:
# -----------------------------
# PydanticAI dependencies
# -----------------------------

# Access to external ressources, including API clients
# No global variables. Keeps API keys secret. 
@dataclass
class Deps:
    brave: BraveSearchClient
    jina: JinaReaderClient
    # Small delay between HTTP requests (to avoid hitting rate limits)
    per_request_sleep_s: float = 0.2

#### Agent with multiple tools

In [13]:
#openai_client = OpenAI()
#openai_client = AsyncOpenAI() # asynchronous OpenAI client for making multiple OpenAI API calls concurrently
model = OpenAIChatModel("gpt-4o-mini")
agent = Agent(
    model,
    deps_type=Deps,
    instructions=(
        "If you need up-to-date web info, call the web_search_and_read tool, "
        "then answer using the returned documents and cite URLs."
    ),
)

In [14]:
# -----------------------------
# Web search tool
# -----------------------------

@agent.tool # PydanticAI decorator
def web_search_and_read(
    ctx: RunContext[Deps],
    query: str,
    num_results: int = 5,
    max_chars_per_doc: int = 12000,
    use_readerlm_v2: bool = True,
) -> WebSearchAndReadOutput:
    """
    Search the web with Brave, then extract readable text from the top URLs using Jina Reader.

    Args:
        query: Web search query.
        num_results: Number of Brave results to fetch.
        max_chars_per_doc: Truncate each extracted document to this many characters.
        use_readerlm_v2: Use Jina's ReaderLM-v2 mode for higher-quality extraction.
    """
    # Load dependencies object (to allow for accessing the Brave and Jina clients)
    deps = ctx.deps

    # Call Brave Search to get web search results 
    # Ensure consistency of url/title/description
    brave_raw = deps.brave.search(query=query, count=num_results)
    search_results = [BraveResult(**r) for r in brave_raw]

    # Lists that will be returned to the agent:
    # - documents: successful extractions from Jina Reader
    # - errors: failures per URL
    documents: List[WebDocument] = []
    errors: List[Dict[str, str]] = []

    # Keep track of URLs we've already processed (to avoid duplicate work)
    seen: set[str] = set()

    # For each search result URL, try to extract text
    for r in search_results:
        url = str(r.url)
        if url in seen: # skip duplicates
            continue
        seen.add(url)

        time.sleep(deps.per_request_sleep_s)

        try:
            resp = deps.jina.read_url(url, use_readerlm_v2=use_readerlm_v2, bypass_cache=True)
            # Process Jina Reader output
            data = resp.get("data") if isinstance(resp, dict) else None
            if not isinstance(data, dict):
                data = resp if isinstance(resp, dict) else {}

            title = (data.get("title") or r.title or url).strip()
            content = (data.get("content") or "").strip()

            # If Jina returned no content, raise error
            if not content:
                raise ValueError("Empty content returned by Jina Reader")

            # Truncate the content so the tool doesn't return huge payloads.
            content = _truncate(content, max_chars_per_doc)
            h = sha256_text(content)

            documents.append(
                WebDocument(
                    url=r.url, # original URL
                    title=title, # title (from Jina, or Brave, or URL)
                    description=r.description, # snippet/description from Brave results
                    text=content, # extracted readable text
                    fetched_at_utc=utc_now_iso(), # timestamp of extraction
                    sha256=h, # hash of text content
                    meta={
                        "jina_status": resp.get("status") if isinstance(resp, dict) else None,
                        "jina_code": resp.get("code") if isinstance(resp, dict) else None,
                    },
                )
            )
            
        # If anything goes wrong for this URL, record the error and continue
        except Exception as e:
            errors.append({"url": url, "error": str(e)})

    # Return everything (query, search results, extracted documents, and errors)
    # as one structured object that the agent can reason over.
    return WebSearchAndReadOutput(
        query=query,
        search_results=search_results,
        documents=documents,
        errors=errors,
    )

In [15]:
def build_deps() -> Deps:
    brave_key = os.environ["BRAVE_SEARCH_API_KEY"]
    jina_key = os.environ.get("JINA_API_KEY")
    session = requests.Session()
    return Deps(
        brave=BraveSearchClient(api_key=brave_key, session=session),
        jina=JinaReaderClient(api_key=jina_key, timeout_s=90),
        per_request_sleep_s=0.25,
    )

In [16]:
result = await agent.run("What's the latest on the EU AI Act enforcement timeline?", deps=build_deps())
print(result.output)

The enforcement timeline for the EU AI Act has several key dates outlined as follows:

1. **Initial Effect**: The Act is set to enter into force on **August 1, 2024**. However, this is the date when the legal framework comes into existence, but its provisions will be rolled out over time.

2. **Phased Implementation**:
   - Full compliance obligations for high-risk AI systems will begin incrementally, with significant milestones spreading through the following years.
   - By **August 2, 2025**, each EU Member State is required to have designated national competent authorities that will oversee the implementation and enforcement of the Act.

3. **Compliance Deadlines**:
   - Different types of AI systems have tailored deadlines to ensure compliance with regulations. The last major deadline is set for **August 2, 2027**, by which time all AI systems, including those used in large-scale IT systems that were placed on the market before this date, must comply with the new regulations.
   - 